## Trial File to make sure repository git pull/commit is working and google solar API testing 

### Importing

In [2]:
import yfinance as yf
import datetime as dt
import pandas as pd
import os
from dotenv import load_dotenv
import googleapiclient.discovery
import requests

## Load Environment Variables

In [3]:
load_dotenv()
GEOCODE_API_KEY = os.getenv("GEOCODING_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("API key not found. Make sure you have a .env file with GOOGLE_API_KEY set.")


## Get Request for Solar Data

### First Attempt: Gemini suggested (Google Solar API)
Generates a "service" to be used with requests - I don't understand this too much, seems to harder to manipulate the request.

In [11]:
# The service and version of the API
api_service_name = "solar"
api_version = "v1"

# The location for which to get solar data
# Replace with the desired latitude and longitude
latitude = 52.267142
longitude = -0.756885

#url = f"https://solar.googleapis.com/v1/buildingInsights:findClosest?key={GOOGLE_API_KEY}&latitude={latitude}&longitude={longitude}"

### Using URL Request (Google Solar API)

In [10]:
import os
import googleapiclient.discovery
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get your API key from the environment variable
api_key = os.getenv("GOOGLE_API_KEY")

# Check if the API key is available
if not api_key:
    raise ValueError("API key not found. Make sure you have a .env file with GOOGLE_API_KEY set.")

# The service and version of the API
api_service_name = "solar"
api_version = "v1"

# The location for which to get solar data
# Replace with the desired latitude and longitude
latitude = 52.2661775
longitude = -0.7577842
#longitude = -0.770228

def get_solar_data2(lat, long):
    building_insights_url = f"https://solar.googleapis.com/v1/buildingInsights:findClosest?location.latitude={lat}&location.longitude={long}&requiredQuality=MEDIUM&key={GOOGLE_API_KEY}"
    data_layers_url = f"https://solar.googleapis.com/v1/dataLayers:get?location.latitude={long}&location.longitude={long}&radius_meters=5&requiredQuality=MEDIUM&key={GOOGLE_API_KEY}"
    b_i_response = requests.get(url=building_insights_url)
    d_l_response = requests.get(url=data_layers_url)
    return b_i_response, d_l_response


if __name__ == "__main__":
    res = get_solar_data2(latitude, longitude)
    print(res[1].text)

{
  "error": {
    "code": 404,
    "message": "Requested entity was not found.",
    "status": "NOT_FOUND"
  }
}



### PVGIS Request
This API might be better than google, as google cannot give the solar flux/energy production from a cell for all of the UK, whereas this should.

In [45]:
#lat & long
latitude = 52.2661775
longitude = -0.7577842

rad_database = "PVGIS-SARAH"

#The peak power output of a solar cell under perfect test conditions, measured in Watts. The value of 400 is an industry standard/average I am using to start off with
peak_power = 0.4

#The system loss, given as a percentage, is the power lost due problems in the system from cables, dirt, inverters etc. another industry average/estimate of 14% used
system_loss = 14.0

pv_technology = "crystSi"

power_url = f"https://re.jrc.ec.europa.eu/api/v5_3/PVcalc?lat={latitude}&lon={longitude}&raddatbase={rad_database}&peakpower={peak_power}&loss={system_loss}&outputformat=json"

In [ ]:
power_response = requests.get(power_url)
print(power_response)
data = power_response.json()
#monthly average energy production in kWh is in data["outputs"]["monthly"][x]["E_m"] where x is the list indice for which month 0-11
#average in the year in kWh is in data["outputs"]["totals"]["fixed"]["E_y"]

{'inputs': {'location': {'latitude': 52.2661775, 'longitude': -0.7577842, 'elevation': 89.0}, 'meteo_data': {'radiation_db': 'PVGIS-SARAH3', 'meteo_db': 'ERA5', 'year_min': 2005, 'year_max': 2023, 'use_horizon': True, 'horizon_db': 'DEM-calculated'}, 'mounting_system': {'fixed': {'slope': {'value': 0, 'optimal': False}, 'azimuth': {'value': 0, 'optimal': False}, 'type': 'free-standing'}}, 'pv_module': {'technology': 'c-Si', 'peak_power': 0.4, 'system_loss': 14.0}, 'economic_data': {'system_cost': None, 'interest': None, 'lifetime': None}}, 'outputs': {'monthly': {'fixed': [{'month': 1, 'E_d': 0.23, 'E_m': 7.01, 'H(i)_d': 0.78, 'H(i)_m': 24.19, 'SD_m': 0.77}, {'month': 2, 'E_d': 0.45, 'E_m': 12.65, 'H(i)_d': 1.44, 'H(i)_m': 40.3, 'SD_m': 2.02}, {'month': 3, 'E_d': 0.85, 'E_m': 26.22, 'H(i)_d': 2.57, 'H(i)_m': 79.75, 'SD_m': 3.35}, {'month': 4, 'E_d': 1.35, 'E_m': 40.37, 'H(i)_d': 4.11, 'H(i)_m': 123.36, 'SD_m': 4.49}, {'month': 5, 'E_d': 1.56, 'E_m': 48.51, 'H(i)_d': 4.84, 'H(i)_m': 150

## Geocoding Request

#### Using a generic geocoding API (worse)

In [ ]:
x = "34 Northampton Road, Earls Barton, UK, NN60HA"
y = "Northampton, UK"
inputted_address = x

geocode_url = f"https://geocode.maps.co/search?q={inputted_address}&api_key={GEOCODE_API_KEY}"

#lat_long_response = requests.get(geocode_url)
#print(lat_long_response.text)

[{"place_id":247867770,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"way","osm_id":991573254,"boundingbox":["52.2660547","52.2661775","-0.7579467","-0.7577842"],"lat":"52.2661775","lon":"-0.7577842","display_name":"Northampton Road, Earls Barton, North Northamptonshire, England, NN6 0HA, United Kingdom","class":"highway","type":"secondary","importance":0.50001}]


#### Using the Google Geocoding API

In [ ]:
x = "34 Northampton Road, Earls Barton, UK, NN60HA"
y = "Northampton, UK"
inputted_address = x

google_geocode_url = f"https://maps.googleapis.com/maps/api/geocode/json?address=0+Northampton+Road,+Earls+Barton,+UK&key={GOOGLE_API_KEY}"

#edit: make the response a json instead of response object to be able to subscript within.

lat_long_response = requests.get(google_geocode_url).json()
print(lat_long_response)  
latlong_dict = lat_long_response["results"][0]["geometry"]["location"]
latitude = latlong_dict["lat"]
longitude = latlong_dict["lng"]

{'results': [{'address_components': [{'long_name': '0', 'short_name': '0', 'types': ['street_number']}, {'long_name': 'Northampton Road', 'short_name': 'Northampton Rd', 'types': ['route']}, {'long_name': 'Earls Barton', 'short_name': 'Earls Barton', 'types': ['locality', 'political']}, {'long_name': 'Northampton', 'short_name': 'Northampton', 'types': ['postal_town']}, {'long_name': 'North Northamptonshire', 'short_name': 'North Northamptonshire', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'England', 'short_name': 'England', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United Kingdom', 'short_name': 'GB', 'types': ['country', 'political']}, {'long_name': 'NN6', 'short_name': 'NN6', 'types': ['postal_code', 'postal_code_prefix']}], 'formatted_address': '0 Northampton Rd, Earls Barton, Northampton NN6, UK', 'geometry': {'location': {'lat': 52.2644415, 'lng': -0.7595162999999999}, 'location_type': 'RANGE_INTERPOLATED', 'viewport': {'no